# Problem Set 2.9: Pivot Tables and Groupby

[Click here to open this notebook in your browser](https://leifwalsh.github.io/data-analysis-problem-sets/lab/index.html?path=2-pandas-basics/2.9-pivot-tables-and-groupby/2.9-pivot-tables-and-groupby.ipynb)

Learn to do grouped aggregations to study cohort behavior or summarize a dataset.

At the end of the [last notebook](../2.8-aggregations/2.8-aggregations.ipynb), we wanted to compute some aggregations (sums, averages, and counts) across different groups. Today, we'll explore the APIs that `pandas` provides for us to do that, since it's so common.

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("standings.csv")
df.head(5)

## Exploring groupby()

First, let's try to sum the wins, losses, and ties separately for each team. We'll build up slowly and inspect the objects we're working with.

First, let's just see what `groupby()` does by itself. We'll ask `pandas` to form groups based on the `team` column:

In [ ]:
df.groupby("team")

....... okay. What is that?

This is something that's hard to figure out by interrogating, so let's just explain it. The [user guide for groupby](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) says a lot, but let's build up to it.

This `DataFrameGroupBy` object is sort of a proxy or view on the original DataFrame, that knows you want to form groups according to the unique values in the `team` column. It has a lot of methods on it that we'll get to, but for one thing, you can iterate over it.

When you do, you get pairs of values, the first is one of the values from the `team` column, and the second is a DataFrame of all the rows that have that value in their `team` column:

In [ ]:
for key, group in df.groupby("team"):
    print(key)

So, if you want to sum up wins per team, you can do this:

In [ ]:
for team, group in df.groupby("team"):
    print(team, group["wins"].sum())

Or you could build your own DataFrame, with `pd.concat()`:

In [ ]:
rows = {}
for team, group in df.groupby("team"):
    rows[team] = group[["wins", "losses", "ties"]].sum()
pd.concat(rows, axis=1)

## Methods on a `DataFrameGroupBy` object

But that's not usually the way you interact with `groupby()`. The `DataFrameGroupBy` object actually behaves a lot like a DataFrame: in particular you can select a column or set of columns with the `[]` indexer:

In [ ]:
df.groupby("team")[["wins", "losses", "ties"]]

It doesn't look like anything changed, but that's actually a new `DataFrameGroupBy` that just has those three columns, but also remembers that you're grouping by `team`.

Now, we get to the fun part. Remember how a `DataFrame` has a `sum()` method (and a lot more aggregation methods)?

In [ ]:
df[["wins", "losses", "ties"]].sum()

A `DataFrameGroupBy` object also has those aggregation methods, but when you call them, it'll apply them separately for each group. This is pretty much the main point of doing `groupby`:

In [ ]:
df.groupby("team")[["wins", "losses", "ties"]].sum()

You can also do this to a single column of a `groupby` result. First, let's see what that object looks like:

In [ ]:
df.groupby("team")["wins"]

We're starting to see a pattern: 1-dimensional objects are some kind of Series, and 2-dimensional objects are some kind of DataFrame. Naturally, aggregating this `SeriesGroupBy` should produce a Series, where the index comes from the grouping column (`team`) and the values are the sums per team:

In [ ]:
df.groupby("team")["wins"].sum()

### Multi-dimensional `groupby()`

Sometimes your categories are identified by more than one dimension (or column).

Suppose that instead of wanting to aggregate things just by `team`, you instead wanted to define your groups by `season` and `division`: that is, for each year, you want to count the number of wins claimed by each division.

Well, you can use `groupby()` with multiple columns too:

In [ ]:
df.groupby(["season", "division"])["wins"].sum()

That's...funny looking. It renders a little better as a DataFrame:

In [ ]:
df.groupby(["season", "division"])["wins"].sum().to_frame()

What's going on here is that the data in this Series is still just the list of win counts. But the index is a new thing: a `MultiIndex` with two levels: `season` and `division`. This is representing a sort of hierarchy in how we would name the thing that identifies one of the values: we need to know both the season and division in order to look up which value we want.

Let's get that Series again:

In [ ]:
yearly_division_wins = df.groupby(["season", "division"])["wins"].sum()
yearly_division_wins

Its values are just the numbers:

In [ ]:
yearly_division_wins.values

But the index is what's interesting:

In [ ]:
yearly_division_wins.index

Anyway, since we wanted to do our aggregation along these two dimensions (season and division), we probably want to look at it as a two-dimensional thing! We can get there with one of pandas's reshaping methods: [`unstack()`](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#stack-and-unstack). 

In [ ]:
yearly_division_wins.unstack()

Great! Now we have a nice table to look at. Let's put it all together in one cell:

In [ ]:
df.groupby(["season", "division"])["wins"].sum().unstack()

Just for fun, let's style it a bit:

In [ ]:
df.groupby(["season", "division"])["wins"].sum().unstack().style.bar(cmap="cividis")

## Pivot Tables

A pivot table might be more familiar to Excel power users. We just saw how to aggregate along two dimensions with `groupby()`. Let's point out the things we needed to specify:

In [ ]:
#           dimensions for aggregation
#               |         |        values to aggregate
#               |         |          |     aggregation to perform
#               |         |          |      |
#               v         v          v      v
df.groupby(["season", "division"])["wins"].sum().unstack()

`pandas` offers us a single function that does all of this in one step: `pivot_table()`:

In [ ]:
#                      values                   dimensions                    aggregation
#                         |                    /          \                       |
#                         |                   /            \                      |
#                         v                  v              v                     v
df.pivot_table(values=["wins"], index=["season"], columns=["division"], aggfunc="sum")

You may have noticed some of those parameters are lists: you can aggregate multiple values per group, or have multiple columns identifying the rows (index) or columns that name each group.

The output of such a thing is pretty complicated to work with, so we won't go further with it, but if you need something like that, you should know `pivot_table` can do it and read the [documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/groupby.html) to understand how to work with it:

In [ ]:
df.pivot_table(values=["wins", "losses", "ties"], index=["season"], columns=["conf", "division"], aggfunc="sum")

## Joining after a `groupby`

We should cover one more thing. Recall that when we talked about `pd.merge()` we also mentioned that there's a `DataFrame.join()` method that basically does the same thing, except it uses the index as the join key instead of a column name by default.

`DataFrame.join()` works great with `groupby()`.

Suppose you want to see each team's percentage contribution of wins per season to its conference. We need to compute the number of wins per season and conference, but then align those back to each row in the original DataFrame so we can compare each team's wins with the total wins in its conference. Let's start by doing that `groupby`:

In [ ]:
df.groupby(["season", "conf"])["wins"].sum()

Now, we want to attach these values back to the original DataFrame. We can do that with `join`, we just need to tell it again which columns it should match to the index of this Series. Since there's already a `wins` column, we also tell `join` to add the column with a suffix on its name:

In [ ]:
df.join(df.groupby(["season", "conf"])["wins"].sum(), on=["season", "conf"], rsuffix="_conf")

Note the new `wins_conf` column on the right.

Finally, we can add the column we want to compute:

In [ ]:
key = ["season", "conf"]
joined = df.join(df.groupby(key)["wins"].sum(), on=key, rsuffix="_conf")
joined["conf_pct_wins"] = 100.0 * joined["wins"] / joined["wins_conf"]
joined

### Groupby vs. Pivot Tables

Okay, but which one should I use?  

* If you're looking for something quick and efficient, `groupby()` is slightly faster for larger datasets

* `groupby()` allows for custom aggregation functions, while `pivot_table()` only supports common aggregation functions

* If you're looking to share your results with a larger group, `pivot_table()` has better formatting and a tabular output
  
* `pivot_table()` has built-in flexibility with parameters like `margins=True` and `fill_value=0`. If you were using groupby, you would need to manage those things yourself

For most use cases, it doesn't matter which one you pick. If you don't use the `columns` parameter in `pivot_table()`, then `groupby()` and `pivot_table()` produce the same data. In fact, pivot tables are essentially defined using groupby!

Remember, these methods are related:

`pivot_table --> groupby + unstack`

and 

`groupby --> pivot_table + stack`

## Exercises 


In [ ]:
# Re-read data just in case
standings = pd.read_csv("standings.csv")
standings

## Example 1

Compute the min and max number of points scored per team.

## Example 2

Count, for each season, how many teams from each division made it to the playoffs.

_Hint: if a team makes it to the playoffs, their `seed` will be non-null._

## Example 3

Which team had the best record each year? Show your findings in a table you would present on a slide.

## Example 4

For each division, which team is most consistently the highest ranked?